In [19]:
#Packages
import pandas as pd
from scipy.integrate import solve_ivp
import numpy as np
from numpy import linalg as LA
from numpy import random
from numpy.random import randn

# Sampling parameters and Calculating $R_1$, $R_2$, $C_1$ and $C_2$
When running this code, you will generate a new dataset. The original dataset used for the main paper figures can be found in the folder "Data_Original".

In [ ]:
##### Functions #####

#Model equations for the unicellular ancestor alone
def ES1(t,f,r1A,r1B,g,m1AB,m1BA):
    #variables
    vA = f[0]
    vB = f[1]
    #equations
    dvA = (r1A)*vA - g*vA**2 - m1AB*vA + m1BA*vB
    dvB = (r1B)*vB - g*vB**2 + m1AB*vA - m1BA*vB
    
    return [dvA,dvB]

#Calculation of the Jacobian matrix eigenvalues for a multicellular mutant attempting to invade.
def EigM(vA,vB,pars):

    r1A,r1B,r2A,r2B,g,m1AB,m1BA,m2AB,m2BA=pars

    M=np.array([[ -(r1A) -g*vA -m1AB, m1BA, 2*r2A, 0],
               [ m1AB, -(r1B) -g*vB -m1BA, 0, 2*r2B],
                [ r1A, 0, - g*vA - m2AB, m2BA],
                [ 0, r1B, m2AB, - g*vB - m2BA]])

    return LA.eig(M)

######## Main code ########

Sample_size=10000

#Migration regimes:
m_value_str=['low','mid','high']
m_range=[1/50,1,50]

#Loop over the different migration regimes
for m_ii in [0,1,2]:
    
    g=1 #\gamma

    #Setting the lists to be filled with sampled data
    R2_l=np.zeros(Sample_size)
    C2_l=np.zeros(Sample_size)
    R1_l=np.zeros(Sample_size)
    C1_l=np.zeros(Sample_size)
    
    qA_l=np.zeros(Sample_size)
    p1A_l=np.zeros(Sample_size)
    p2A_l=np.zeros(Sample_size)
    
    m1AB_l=np.zeros(Sample_size)
    m1BA_l=np.zeros(Sample_size)
    m2AB_l=np.zeros(Sample_size)
    m2BA_l=np.zeros(Sample_size)
    
    
    m_l=np.zeros(Sample_size)
    
    r1A_l=np.zeros(Sample_size)
    r2A_l=np.zeros(Sample_size)
    r1B_l=np.zeros(Sample_size)
    r2B_l=np.zeros(Sample_size)
    
    w_l=np.zeros(Sample_size)
    count=-1

    #Loop until we have enough samples
    while count<=Sample_size-2:
        count+=1

        #Total growth rate: 
        r=1
        #Total migration rate: (depends on the migration regime)
        m=m_range[m_ii]

        #Randomly sample the growth rates
        rps=[np.random.uniform() for i in range(4)]
    
        r1A=r*rps[0]/sum(rps)
        r1B=r*rps[1]/sum(rps)
    
        r2A=r*rps[2]/sum(rps)
        r2B=r*rps[3]/sum(rps)

        #Check that there are no direct benefits
        if r1A<r2A or r1B<r2B:
            count-=1
            continue
        
        r1A_l[count]=r1A
        r1B_l[count]=r1B
        r2A_l[count]=r2A
        r2B_l[count]=r2B

        #Randomly sample the migration rates
        
        mps=[np.random.uniform() for i in range(4)]
    
        m1AB=m*mps[0]/sum(mps)
        m1BA=m*mps[1]/sum(mps)
    
        m2AB=m*mps[2]/sum(mps)
        m2BA=m*mps[3]/sum(mps)
        
        m1AB_l[count]=m1AB
        m1BA_l[count]=m1BA
        m2AB_l[count]=m2AB
        m2BA_l[count]=m2BA
        
        m=m1AB+m1BA+m2AB+m2BA
     
        m_l[count]=m
        
        #Simulate the ancestor until equilibrium
        ts0=np.linspace(0,3000000,1000)
        q0 = [(r1A)/g,(r1B)/g]
        qsol = solve_ivp(ES1,(ts0[0],ts0[-1]),q0,args=(r1A,r1B,g,m1AB,m1BA), dense_output=True,method='LSODA')
        qs= qsol.sol(ts0)
        qs_0=qs[:,-1]
    
        #Calculate the equilibrium values for
        vA=qs_0[0] # Number of ancestral cells in A
        vB=qs_0[1] # Number of ancestral cells in B
        qA= vA/(vA+vB) # Proportion of ancestral cells in A
        qB=1-qA        # Proportion of ancestral cells in B
    
    
        #Calculate the maximum eigenvalue of the Jacobian matrix for the multicellular mutant attempting to invade.
        pars=[r1A,r1B,r2A,r2B,g,m1AB,m1BA,m2AB,m2BA]
        e_val, e_vec = EigM(vA,vB,pars)
        i_max=list(e_val).index(max(e_val))
        ev=e_val[i_max]

        #Calculate the:
        uAe=abs(e_vec[:,i_max][0]) # Number of 2+1 single cells in A
        uBe=abs(e_vec[:,i_max][1]) # Number of 2+1 single cells in B
    
        UAe=abs(e_vec[:,i_max][2]) # Number of 2+1 groups in A
        UBe=abs(e_vec[:,i_max][3]) # Number of 2+1 groups in B
    
        p1A=uAe/(uAe+uBe) # Proportion of 2+1 single cells in A
        p1B=1-p1A # Proportion of 2+1 single cells in B
        p2A=UAe/(UAe+UBe) # Proportion of 2+1 groups in A
        p2B=1-p2A # Proportion of 2+1 groups in B
        
        # Record the values
        qA_l[count]=qA
        p1A_l[count]=p1A
        p2A_l[count]=p2A
        
        # Calculate R1, R2, C1, C2
        R1= (p1A*r1A + p1B*r1B)/(qA*r1A + qB*r1B)
        R2= (p2A*r2A + p2B*r2B)/(qA*r1A + qB*r1B)
        C1= (p1A*qA + p1B*qB)/(qA**2 + qB**2)
        C2= (p2A*qA + p2B*qB)/(qA**2 + qB**2)
        
        R1_l[count]=R1
        R2_l[count]=R2
        C1_l[count]=C1
        C2_l[count]=C2
        
    # Save the results into a dataframe
    d_l={"R1": R1_l,"C1": C1_l,"R2": R2_l,"C2": C2_l,"m": m_l,"m1AB":m1AB_l,"m1BA":m1BA_l,"m2AB":m2AB_l,"m2BA":m2BA_l,
        "qA":qA_l,"p1A":p1A_l,"p2A":p2A_l,"r1A":r1A_l,"r1B":r1B_l,"r2A":r2A_l,"r2B":r2B_l,"win":w_l}
    
    df_l=pd.DataFrame(d_l)
    
    df_l.to_csv('Data/Figure 2/Fig2-multicell-'+m_value_str[m_ii]+'.csv',index = False)

# Simulating the invasion to obtain the long term ecological outcome
When running this code, you will generate a new dataset. The original dataset used for the main paper figures can be found in the folder "Data_Original".

In [ ]:



# Model equations for the unicellular ancestor alone
def Simulation_Uni(t, f, r1A, r1B, gamma, m1AB, m1BA):
    yA, yB = f
    dyA_dt = r1A * yA - gamma * yA * yA - m1AB * yA + m1BA * yB
    dyB_dt = r1B * yB - gamma * yB * yB - m1BA * yB + m1AB * yA
    return [dyA_dt, dyB_dt]


# Full model equations for 2+1
def Simulation_MC(t, f, r1A, r1B, r2A, r2B, gamma, m1AB, m1BA, m2AB, m2BA):
    yA, yB, x1A, x1B, x2A, x2B = f

    TA = yA + x1A + 2 * x2A
    TB = yB + x1B + 2 * x2B

    # Ancestor
    dyA_dt = r1A * yA - gamma * TA * yA - m1AB * yA + m1BA * yB
    dyB_dt = r1B * yB - gamma * TB * yB - m1BA * yB + m1AB * yA

    # Single-cell multicellular mutant
    dx1A_dt = 2 * r2A * x2A - r1A * x1A - gamma * TA * x1A - m1AB * x1A + m1BA * x1B
    dx1B_dt = 2 * r2B * x2B - r1B * x1B - gamma * TB * x1B - m1BA * x1B + m1AB * x1A

    # Two-cell multicellular mutant
    dx2A_dt = r1A * x1A - gamma * TA * x2A - m2AB * x2A + m2BA * x2B
    dx2B_dt = r1B * x1B - gamma * TB * x2B - m2BA * x2B + m2AB * x2A

    return [dyA_dt, dyB_dt, dx1A_dt, dx1B_dt, dx2A_dt, dx2B_dt]

# Classification: Dominance or Coexistence
def Classify_Eco_Composition(yA, yB,x1A, x1B, x2A, x2B, tol=1e-6):
    y_sum = max(0.0, yA) + max(0.0, yB)
    x_sum = max(0.0,x1A) + max(0.0,x1B) + max(0.0,2*x2A) + max(0.0,2*x2B)

    if y_sum > tol and x_sum > tol:
        return 1 #coexistence
    
    elif y_sum < tol and x_sum > tol:
        return 2 #multicell dominance
    
    elif y_sum > tol and x_sum < tol:
        return 0 #unicell dominance
    
    else:
        return -1 #extinction (never observed)


def classify_fixed_or_cycle(x_tail, tol=1e-6):

    max_diff=0.0
    for i in range(x_tail.shape[0]):
        x_i_tail = x_tail[i, :]
        diff = np.max(x_i_tail) - np.min(x_i_tail)

        if diff > max_diff:
            max_diff = diff

    if max_diff < tol:
        return 1 # fixed point
    else:
        return 2 # limit cycle (never observed)


types=["low","mid","high"]
for type in types:
    # Load the data
    df = pd.read_csv(f'Data/Figure 2/Fig2-multicell-{type}.csv')

    # Select all four m's and all four r's
    m_cols = ['m1AB', 'm1BA', 'm2AB', 'm2BA']
    r_cols = ['r1A', 'r1B', 'r2A', 'r2B']
    gamma = 1.0

    Eco_O = []  # Ecological outcomes
    # # 0= Fixed point dominance of unicellularity, 1 = Fixed point coexistence, 2 = Fixed point dominance of multicellularity, 
    # # 3= Limit cycle coexistence, 4 = Limit cycle dominance of multicellularity.

    for idx in range(10000): #Only look at the first 10000 points
        # Load row values
        row = df.iloc[idx]
        m1AB, m1BA, m2AB, m2BA = row[m_cols].values
        r1A, r1B, r2A, r2B = row[r_cols].values

        y0 = [r1A / gamma, r1B / gamma]

        t0, tf = 0.0, 1e7

        ysol = solve_ivp(
            Simulation_Uni, (t0, tf), y0,
            args=(r1A, r1B, gamma, m1AB, m1BA), method='LSODA',dense_output=True
        )
        yA, yB = ysol.sol(tf) 

        x0 = [yA, yB, 1e-4, 1e-4, 0.0, 0.0]
        xsol = solve_ivp(
            Simulation_MC, (t0, tf), x0,
            args=(r1A, r1B, r2A, r2B, gamma, m1AB, m1BA, m2AB, m2BA), method='LSODA', dense_output=True
        )

        t_tail=np.linspace(tf-1000, tf, 1000)
        x_tail=xsol.sol(t_tail)
        yA, yB, x1A, x1B, x2A, x2B = x_tail[:,-1]
        
        eco=Classify_Eco_Composition(yA, yB,x1A, x1B, x2A, x2B, tol=1e-6)
        cyc=classify_fixed_or_cycle(x_tail, tol=1e-6)

        if eco== 1:
            if  cyc== 1:
                Eco_O.append(1)
            else:
                Eco_O.append(3)

        elif eco== 2:
            if cyc == 1:
                Eco_O.append(2)
            else:
                Eco_O.append(4)

        elif eco== 0:
            Eco_O.append(0)
        
        else:
            Eco_O.append(-1) #extinction (never observed)


    df["Eco_O"] = Eco_O
    df.to_csv(f"Data/Figure 2/Fig2-multicell-{type}-Eco.csv", index=False)